## BySearch
Пакет bysearch реализует легкие и эффективные пайплайны semantic search поисковиков с помощью простого и доступного API, объединяя современные векторные хранилища с легковесными открытыми языковыми моделями.

Моя цель - открытое, доступное и простое во всех смыслах решение, с помощью которого даже незнакомые с Deep learning'ом разработчики могли бы создавать и внедрять в свои проекты semantic search движки для любых языков (по крайней мере в том случае, если для выбранного языка существует хотя бы одна, даже самая маленькая, NLP модель). 

In [1]:
from datasets import load_dataset

from bysearch import Engine
from bysearch.pipelines import HuggingFacePipeline, ONNXPipeline
from bysearch.backends import DatasetBackend, PineconeBackend, ChromaBackend 

c:\Users\tihol\Projects\PyProjects\BySearch\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Разберем работу пакета на примере создания поисковика для беларускамоўных текстов.

Базовым классом пакета является класс Engine, он организует все потоки данных, начиная с голого текста и заканчивая хранилищем векторов. Engine руководит другими модулями, которые осуществляют обработку и превращение текста в эмбеддинги (векторное представление текста), а также реализуют операции над выбранным векторным хранилищем. Класс Engine реализует единый простой API, который на данный момент поддерживает операции добавления коллекции текстов, удаления текстов по ID и поиска текста по промпту. 

Для работы Engin'у необходимы два компонента: 
1. DataBackend, который непосредственно реализует общение с выбранной базой данных.
2. EmbeddingPipeline, который реализует алгоритмы генерации эмбеддингов текстов.

Добавляемый текст должен храниться в Pandas DataFrame или в Hugging Face Dataset.

Взглянем на DataBackend'ы. На данный момент поддерживаются бекенды для взаимодействия с хранилищами Pinecone и Chroma, а также простой локальный бекенд, реализуемый на Hugging Face Dataset'ах. Общими обязательнами параметрами при инициализации любых классов DataBackend являются text_column_name, который должен содержать имя поля, в котором текст содержится в добавляемых коллекциях и в котором текст будет содержаться в векторном хранилище, а также id_column_name, которое содержит имя поля с уникальным идентификатором каждого текста в добавляемой коллекции и в векторном хранилище. Остальные параметры инициализации каждого отдельного бекенда служат для подключения к соответствующей ему базе данных. DataBackend'ы обеспечивают подключение к уже существующим хранилищам или автоматическое создание новых в том случае, если хранилища с указанным именем не сущесвтует. 

In [2]:
# Простой локальный бекенд, хранится в оперативной памяти в рамках сессии Python,
# не поддерживает удаление, не отслеживает дубликаты текстов.
# Рекомендуется использовать исключительно при тестировании.
# backend = DatasetBackend(
#     text_column_name='text', 
#     id_column_name='id'
# )

# Бекенд реализующий подключение и общение с хранилищем Pinecone.
# Pinecone является платным закрытым облачным хранилищем, предоставляющим доступ к API по ключу.
# backend = PineconeBackend(
#     text_column_name='text', 
#     id_column_name='id', 
#     api_key='your key', 
#     environment ='gcp-starter', 
#     index_name='your index name'
# )

# Бекенд реализующий подключение и общение с хранилищем Chroma.
# Chroma является гибким открытым хранилищем.
# Поддерживает локальное хранение в оперативной памяти, на диске, а также на сервере.
backend = ChromaBackend(
    text_column_name='text', 
    id_column_name='id', 
    type='persistent', 
    collection_name='by-embeddings'
)

Перейдем к EmbeddingPipelin'ам. На данный момент реализована поддержка моделей из крупнейшего хаба моделей Hugging Face, реализована  поддержка моделей в формате ONNX, а также конвертер моделей Hugging Face в формат ONNX для быстрой обработки текстов. 

Создадим пайплайн, использующий маленькую RoBERT'у, обученную для беларускай мовы. 

Выбор модели зависит от используемых языков в корпусе текстов, который будет обрабатываться моделью. 

In [3]:
# HuggingFacePipeline реализует генерацию эмбеддингов текста
# с помощью любых языковых моделей-трансформеров из хаба Hugging Face.
# Рекомендуется использовать только в процессе тестирования.
# pipeline = HuggingFacePipeline(
#     model='KoichiYasuoka/roberta-small-belarusian', 
#     max_context_length=127
# )

# ONNXPipeline реализует генерацию эмбеддингов текста
# с помощью любых языковых моделей-трансформеров в формате ONNX.
# Рекомендуется использовать для долговременной поддержки поискового движка,
# поскольку модели в формате ONNX оптимизированы для быстрой работы.
# Для данного пайплайна реализован метод from_hugging_face,
# который конвертирует модели Hugging Face в формат ONNX.
pipeline = ONNXPipeline.from_hugging_face(
    model='KoichiYasuoka/roberta-small-belarusian', 
    onnx_save_path='onnx\\model.onnx', 
    max_context_length=127, 
    dummy_input=['What is Lorem Ipsum?Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industrys standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum. Why do we use it?It is a long established fact that a reader will be distracted by the readable content of a page when looking at its layout. The point of using Lorem Ipsum is that it has a more-or-less normal distribution of letters, as opposed to using, making it look like readable English. Many desktop publishing packages and web page editors now use Lorem Ipsum as their default model text, and a search for will uncover many web sites still in their infancy. Various versions have evolved over the years, sometimes by accident, sometimes on purpose (injected humour and the like).']
)

Some weights of RobertaModel were not initialized from the model checkpoint at KoichiYasuoka/roberta-small-belarusian and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Подготовим данные из корпуса беларуских текстов и соберем Engine. 

In [4]:
# Загружаем данные в формате Hugging Face Dataset и добавляем поле ID.
# Уникальным идентификатором в текста может быть любое уникальное поле из его metadata,
# однако некоторые хранилища, в частности Pinecone, имеют ограничения по длине поля уникального идентификатора.
dataset = load_dataset('mc4', 'be', split='validation')
dataset = dataset.add_column('id', list(range(len(dataset))))
# Собираем Engine из выбранных пайплайна и бекенда,
# а также передаем загружаемый корпус текстов в параметр dataset.
# Параметр dataset не является обязательным при инициализации Engine, 
# корпус текстов может быть загружен отдельно с помощью метода upsert.
search = Engine(dataset=dataset, pipeline=pipeline, backend=backend)

Parameter 'function'=<function Engine.load_dataset.<locals>.<lambda> at 0x00000286BDF52A20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 1712/1712 [00:42<00:00, 39.98 examples/s]


Осуществим поиск по запросу. Результаты поиска возвращаются в Pandas DataFrame в формате (score, поле уникального идентификатора, текст, все остальные поля).

In [5]:
rez = search.search('аповесць беларускага пісьменніка Уладзіміра Караткевіча', verbose=False)
rez

,score,id,text,timestamp,url
0,6.785421,901,﻿ Стаўленне да вайны — характарыстыка духоўнас...,2020-08-04T17:20:46Z,http://zviazda.by/be/news/20150508/1431035548-...
1,6.891847,77,Мікалай Якаўлевіч Нікіфароўскі — Вікіпедыя\nМі...,2020-08-03T21:55:48Z,https://be.m.wikipedia.org/wiki/%D0%9C%D1%96%D...
2,7.098675,344,"Арлоў Уладзімір, Айчына, частка першая - Белар...",2019-11-12T09:06:47Z,http://ww.w.kamunikat.org/katalohbht.html?pub_...
3,7.118645,1473,"""Беларусь 3"" ушануе памяць Валерыя Чкалава | Б...",2019-02-24T06:42:12Z,http://3belarus.by/be/news/belarus-3-ushanue-p...
4,7.238077,506,Прэзентацыя кнігі «Хуш Килесез — калі ласка!: ...,2019-02-17T06:57:53Z,http://mininform.gov.by/news/stuzhka-navin/pre...


Удалим полученные результаты из хранилища и повторим поиск по идентичному запросу.

Предупреждение: Облачный бекенд Pinecone имеет некоторую задержку обновления хранилища, по этой причине результаты операций с хранилищем Pinecone могут быть доступны не сразу.

In [6]:
search.delete(rez['id'].tolist())
rez = search.search('аповесць беларускага пісьменніка Уладзіміра Караткевіча', verbose=False)
rez

,score,id,text,timestamp,url
0,7.278406,408,Аляксей Белы адзначае юбілей\nГалоўная » Навін...,2020-03-29T15:38:08Z,https://lit-bel.org/news/Alyaksey-Beli-adznach...
1,7.476275,1358,«Дарогамі Максіма Танка»\nОпубликовано: 18.09....,2018-03-23T16:48:12Z,https://www.sb.by/articles/darogam-maks-ma-tan...
2,7.612226,544,"Творчая сустрэча «Родны край, я цябе апяваю»\n...",2019-12-12T14:06:21Z,http://kb.brl.by/index.php/home?view=featured
3,7.784564,120,Прэм'ера ''Смешныя людзі'' - 12.10.2018 - Наци...,2018-10-18T09:00:55Z,https://www.kvitki.by/rus/bileti/teatr/drama/p...
4,7.789008,56,Вязень лагера Дахау | Дзяннiца\nГлавная Вязень...,2017-01-17T09:03:39Z,http://dzyannica.by/node/11144


Загрузим в хранилище копию корпуса текстов.

In [7]:
# Создаем копию данных в формате Hugging Face Dataset и добавляем поле ID.
new_dataset = load_dataset('mc4', 'be', split='validation')
new_dataset = new_dataset.add_column('id', list(range(len(dataset))))
# Метод upsert осуществляет обновление или вставку данных из параметра dataset в хранилище. 
search.upsert(dataset=new_dataset)
rez = search.search('аповесць беларускага пісьменніка Уладзіміра Караткевіча', verbose=False, k=10)
rez

Map: 100%|██████████| 1712/1712 [00:41<00:00, 41.22 examples/s]


,score,id,text,timestamp,url
0,6.629228,1656,﻿ Новыя паступленні музея імя Суворава | А. М....,2017-08-18T16:34:45Z,http://ikobrin.ru/martinov-52.php
1,6.785421,901,﻿ Стаўленне да вайны — характарыстыка духоўнас...,2020-08-04T17:20:46Z,http://zviazda.by/be/news/20150508/1431035548-...
2,6.891847,77,Мікалай Якаўлевіч Нікіфароўскі — Вікіпедыя\nМі...,2020-08-03T21:55:48Z,https://be.m.wikipedia.org/wiki/%D0%9C%D1%96%D...
3,7.098675,344,"Арлоў Уладзімір, Айчына, частка першая - Белар...",2019-11-12T09:06:47Z,http://ww.w.kamunikat.org/katalohbht.html?pub_...
4,7.118645,1473,"""Беларусь 3"" ушануе памяць Валерыя Чкалава | Б...",2019-02-24T06:42:12Z,http://3belarus.by/be/news/belarus-3-ushanue-p...
5,7.238077,506,Прэзентацыя кнігі «Хуш Килесез — калі ласка!: ...,2019-02-17T06:57:53Z,http://mininform.gov.by/news/stuzhka-navin/pre...
6,7.278406,408,Аляксей Белы адзначае юбілей\nГалоўная » Навін...,2020-03-29T15:38:08Z,https://lit-bel.org/news/Alyaksey-Beli-adznach...
7,7.476275,1358,«Дарогамі Максіма Танка»\nОпубликовано: 18.09....,2018-03-23T16:48:12Z,https://www.sb.by/articles/darogam-maks-ma-tan...
8,7.612226,544,"Творчая сустрэча «Родны край, я цябе апяваю»\n...",2019-12-12T14:06:21Z,http://kb.brl.by/index.php/home?view=featured
9,7.784564,120,Прэм'ера ''Смешныя людзі'' - 12.10.2018 - Наци...,2018-10-18T09:00:55Z,https://www.kvitki.by/rus/bileti/teatr/drama/p...


Как можно убедиться, удаленные тексты вновь появились в хранилище, а хранимые тексты избежали дублирования.